## Get data from crossref

We are going to explore cross_ref which available through an API

Read documentation:
https://api.crossref.org/swagger-ui/index.html#/Works/get_works

Crossref has a specific R package to query the API, which we are going to use today to collect some data
Try the code chunck below, but replacing
(i) the query for the specific month and year assigned to you in class
(ii) increase the limit of the number of paper returned to 400
Running these queries take longer since we are waiting a response from a server
(iii) Check the first 10 rows in the dataframe
(iv) Add your email address to the polite and enter it at the end of query_full

In [163]:
#!pip install pandas requests beautifulsoup4
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [164]:
api_first = "http://api.crossref.org/works?query="
query = "artificial+neural+network"
filter = "&filter="
funder = "has-funder:true,"
from_date = "from-pub-date:2016-01-01,"
to_date = "until-pub-date:2016-01-31"
limit = "&rows=400"
polite = "&mailto=__"

# Put the query together and call it
query_full = api_first + query + filter + funder + from_date + to_date + limit + polite
fetch = requests.get(query_full)

# Transform the response into a readable component
response_content = fetch.content
response_text = response_content.decode('utf-8')
return_all = json.loads(response_text)

# Extract the relevant data and convert it to a DataFrame
ANN_data = pd.DataFrame(return_all["message"]["items"])

(i) Check names of the columns available

Index(['indexed', 'reference-count', 'publisher', 'license', 'funder',
       'content-domain', 'short-container-title', 'published-print', 'DOI',
       'type', 'created', 'page', 'source', 'is-referenced-by-count', 'title',
       'prefix', 'volume', 'author', 'member', 'reference', 'container-title',
       'link', 'deposited', 'score', 'resource', 'issued', 'references-count',
       'URL', 'ISSN', 'issn-type', 'subject', 'published', 'issue',
       'update-policy', 'published-online', 'language', 'journal-issue',
       'alternative-id', 'assertion', 'abstract', 'article-number', 'archive',
       'subtitle', 'publisher-location', 'event', 'relation'],
      dtype='object')


Create a new dataset from the crossref data collected to look at the top funders within those 400 first data point collected
(1) keep only the columns 'container-title', 'DOI', 'title', 'funder'
(2) Rename the DOI column to DOI_papers
(3) separate funders for each funder taking only one row (using 'unnest' of the purr package in tidyr)
(4) View the top 20 rows of the dataframe to see whether things have worked how you intended to

In [167]:


# # Select columns and rename
ANN_funder = ANN_data[['container-title', 'DOI', 'title', 'funder']]
ANN_funder = ANN_funder.rename(columns={'DOI': 'DOI_papers'})

#this is a test one
first_row_data = ANN_funder.at[0, 'funder']
first_dict = first_row_data[0]  # Access the first element in the list
doi = first_dict.get('DOI')
name = first_dict.get('name')
print(doi, name)


10.13039/501100003711 Ministry of Science and Technology


Clean the column names coming from the nested funder dataset:
(1) on top of the columns to keep before, keep 'name' and 'DOI_paper'
(2) rename 'name' to a more appropriate name (as this is specific to the funder)
(3) rename 'DOI' to a more appropriate name (as this is specific to the funder)

In [168]:
def extract_values(row):
    if row:  # Check if the row is not empty
        first_dict = row[0]  # Access the first element in the list
        doi = first_dict.get('DOI')
        name = first_dict.get('name')
        return doi, name
    return None, None


ANN_funder['DOI_funder'], ANN_funder['funder_name'] = zip(*ANN_funder['funder'].apply(extract_values))

Clean the column names coming from the nested funder dataset:
(1) on top of the columns to keep before, keep 'name' and 'DOI_paper'
(2) rename 'name' to a more appropriate name (as this is specific to the funder)
(3) rename 'DOI' to a more appropriate name (as this is specific to the funder)

Create a summary of the funder data, to identify the top funder in your specific month
The resulting dataframe will be arranged in descending order
display the top three answers (these will be reported in class)

In [169]:
# Group by 'funder_name' and count occurrences, then sort in descending order


# Display the top 3 results



                                      funder_name  count
149  National Natural Science Foundation of China     77
145                 National Institutes of Health     18
43                Deutsche Forschungsgemeinschaft     12
